In the modular approach I will create separate files for different model types. This one will be for beta_stack_model_sil_credo_score

# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## calculate_gini

In [2]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_periodic_gini_prod_ver_trench_dimfact

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """
    Calculate Gini coefficient with proper handling of edge cases
    
    Returns np.nan when:
    - Fewer than 2 observations
    - No positive labels (sum of labels = 0)
    """
    n = len(scores)
    if n < 2:
        return np.nan
    
    label_sum = np.sum(labels)
    
    # Handle case where no positive labels exist (all zeros)
    # This prevents division by zero warning
    if label_sum == 0:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * label_sum)
    return gini


def calculate_periodic_gini_prod_ver_trench_dimfact(df, score_column, label_column, namecolumn, 
                                        data_selection_column=None,
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        ostype_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients and return Power BI-friendly long format
    with fact and dimension tables.
    
    Returns:
    - fact_table: Long format with one row per segment per period
    - dimension_table: Unique segment combinations for filtering
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    data_selection_column: (optional) name of column for data selection (Test/Train)
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    ostype_column: (optional) name of column for OS type
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'data_selection': data_selection_column,
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'ostype': ostype_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if data_selection_column:
        segment_columns.append(('DataSelection', data_selection_column))
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    if ostype_column:
        segment_columns.append(('OSType', ostype_column))
    
    # Generate all possible combinations of segment columns
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini_value')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        else:
            weekly_gini['distinct_accounts'] = None
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini_value')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        else:
            monthly_gini['distinct_accounts'] = None
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        gini_results['data_selection'] = metadata.get(data_selection_column, None) if data_selection_column else None
        gini_results['model_version'] = metadata.get(model_version_column, None) if model_version_column else None
        gini_results['trench_category'] = metadata.get(trench_column, None) if trench_column else None
        gini_results['loan_type'] = metadata.get(loan_type_column, None) if loan_type_column else None
        gini_results['loan_product_type'] = metadata.get(loan_product_type_column, None) if loan_product_type_column else None
        gini_results['ostype'] = metadata.get(ostype_column, None) if ostype_column else None
        
        all_results.append(gini_results)
    
    # Combine all results
    fact_table = pd.concat(all_results, ignore_index=True)
    
    # Create dimension table (unique segment combinations for filtering)
    dimension_table = fact_table[['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                   'trench_category', 'loan_type', 'loan_product_type', 'ostype']].drop_duplicates().reset_index(drop=True)
    dimension_table['segment_id'] = range(len(dimension_table))
    
    # Add segment_id to fact table
    fact_table = fact_table.merge(dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 
                                                     'data_selection', 'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']], 
                                  on=['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                      'trench_category', 'loan_type', 'loan_product_type', 'ostype'], 
                                  how='left')
    
    # Reorder columns in fact table
    fact_table = fact_table[['segment_id', 'start_date', 'end_date', 'period', 'gini_value', 'distinct_accounts',
                             'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 'trench_category', 
                             'loan_type', 'loan_product_type', 'ostype']]
    
    # Reorder columns in dimension table
    dimension_table = dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 
                                        'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']]
    
    return fact_table, dimension_table


# Usage:
# fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     data_selection_column='Data_selection',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     ostype_column='osType',
#     account_id_column='digitalLoanAccountId'
# )
# 
# # In Power BI:
# # 1. Import fact_table and dimension_table
# # 2. Create relationship: dimension_table[segment_id] -> fact_table[segment_id]
# # 3. Use dimension table columns as filters (including ostype)
# # 4. Create DAX measures:
# #    - Gini Measure = AVERAGE(fact_table[gini_value])
# #    - Account Count = SUM(fact_table[distinct_accounts])
# # 5. Use start_date, end_date, period for time-based analysis

# update_tables

In [4]:

def update_tables(fact_table: pd.DataFrame, dimension_table: pd.DataFrame, model_name: str, product: str) -> tuple:
    """
    Updates fact_table and dimension_table:
    - Sets 'Model_display_name' to the given model_name
    - Replaces NaN values in specified columns with 'Overall'
    
    Returns:
        Updated fact_table and dimension_table as a tuple
    """
    # Columns where missing values should be replaced
    cols_to_replace = ['model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']
    
    # Update fact_table
    fact_table['Model_display_name'] = model_name
    fact_table['Product_Category'] = product
    fact_table[cols_to_replace] = fact_table[cols_to_replace].fillna('Overall')
    
    # Update dimension_table
    dimension_table['Model_display_name'] = model_name
    dimension_table['Product_Category'] = product
    dimension_table[cols_to_replace] = dimension_table[cols_to_replace].fillna('Overall')
    
    return fact_table, dimension_table


In [5]:
facttable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
dimtable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"

# beta_stack_model_sil_credo_score 

## FPD0 

## Test 

In [6]:
sq = f"""with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
        deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
      coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and flg_mature_fpd0 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()


The shape of the dataframe downloaded is:	 (105211, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3777875,00d9749b-a742-4107-927f-19e3f7a29a25,60837778750011,beta_stack_model_sil,0.0941,"{""s_apps_score"": 0.5395466277027092, ""s_credo_...",2025-10-29 16:46:34,2025-10-29,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,3734637,00f5eb46-dde2-47da-a884-74dcf0cb43b6,60837346370017,beta_stack_model_sil,0.1119,"{""s_apps_score"": 0.48854027267494937, ""s_credo...",2025-10-10 10:03:34,2025-10-10,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
2,3547013,01306bac-1cb3-4d9e-8c78-08abd68f801e,60835470130016,beta_stack_model_sil,0.2076,"{""s_apps_score"": 0.5580538639009095, ""s_credo_...",2025-07-09 14:43:54,2025-07-09,2025-07,Prod,1,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3729769,0161d70a-0028-4611-9f1c-0ad612f53522,60837297690013,beta_stack_model_sil,0.1765,"{""s_apps_score"": 0.557188372871702, ""s_credo_s...",2025-10-07 16:39:03,2025-10-07,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
4,3181470,01d6f779-c51c-457f-8eb1-8fef7ff0d1be,60831814700027,beta_stack_model_sil,0.1285,"{""s_apps_score"": 0.4277879213996953, ""s_credo_...",2025-06-20 10:27:16,2025-06-20,2025-06,Prod,0,1,SIL Competitor,v1,Trench 3,Appliance,android


In [7]:
df1 = dfd.copy()

### Train 

In [8]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
        deviceOs osType,
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and flg_mature_fpd0 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (225063, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3206286,6b2d3c9c-efd4-4d62-a4d3-db28826ffd7a,60832062860017,beta_stack_model_sil,0.050984,"{""sb_demo_score"": 0.49638812817771, ""apps_scor...",2025-11-15 10:23:43,2025-01-21,2025-11,Dev_Test,0,1,SIL-Instore,v2,Trench 1,Appliance,android
1,2788309,08338289-ce5c-42f6-8b66-f08b7f2c3a77,60827883090018,beta_stack_model_sil,0.101556,"{""sb_demo_score"": 0.5035560791661472, ""apps_sc...",2024-08-24 18:29:55,2024-08-24,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Mobile,android
2,2790617,0b1823b5-68c1-47bf-a361-9cc0dbd5ccd4,60827906170011,beta_stack_model_sil,0.157484,"{""sb_demo_score"": 0.4758191497466376, ""apps_sc...",2024-08-25 14:21:12,2024-08-25,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
3,2747589,14127d8a-7adb-4af9-b727-732203642a9c,60827475890016,beta_stack_model_sil,0.105954,"{""sb_demo_score"": 0.722511249740164, ""s_credo_...",2024-08-12 17:30:03,2024-08-12,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,ios
4,2789662,1495bf35-4681-4639-92b0-dd1bf0467bdd,60827896620016,beta_stack_model_sil,0.177820,"{""sb_demo_score"": 0.3092516091552933, ""apps_sc...",2024-08-25 11:03:35,2024-08-25,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android


In [9]:
df2 = dfd.copy()

### Concat 

In [10]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (268212, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3777875,00d9749b-a742-4107-927f-19e3f7a29a25,60837778750011,beta_stack_model_sil,0.0941,"{""s_apps_score"": 0.5395466277027092, ""s_credo_...",2025-10-29 16:46:34,2025-10-29,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,3756414,06e87962-2e6e-4597-82e6-be62cc0cb012,60837564140012,beta_stack_model_sil,0.0949,"{""s_apps_score"": 0.5303339598591643, ""s_credo_...",2025-10-20 13:39:22,2025-10-20,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3788492,0870b2d5-5ea0-46b4-9742-29fb91931432,60837884920019,beta_stack_model_sil,0.1446,"{""s_apps_score"": 0.5030790708952588, ""s_credo_...",2025-11-03 11:16:35,2025-11-03,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
3,3757092,0eecd98a-2246-43aa-8fe5-3254c7272827,60837570920018,beta_stack_model_sil,0.0967,"{""s_apps_score"": 0.3741121261656919, ""s_credo_...",2025-10-20 17:00:03,2025-10-20,2025-10,Prod,0,1,SIL ZERO,v1,Trench 2,Appliance,android
4,3762258,12e8ae4e-1c1f-49a7-a91b-b7dc5471ff5e,60837622580017,beta_stack_model_sil,0.1360,"{""s_apps_score"": 0.6035900860877075, ""s_credo_...",2025-10-23 10:21:01,2025-10-23,2025-10,Prod,0,1,SIL ZERO,v1,Trench 2,Appliance,android


In [11]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [12]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffpd0', 
    'FPD0',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
     ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

In [13]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (50242, 17)
The shape of the dimension table is:	 (1305, 12)


In [14]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=436bac02-b1ca-4511-acc7-0def98b2749f>

In [15]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=aaa7e0a3-182a-4762-b22e-ba5dd98241a0>

## FPD10 

## Test 

In [16]:
sq = f"""with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
        deviceOs osType,
    
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd10 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()


The shape of the dataframe downloaded is:	 (102892, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3510813,0013d101-ba98-4db0-8c7a-197f653d36be,60835108130017,beta_stack_model_sil,0.1190,"{""s_apps_score"": null, ""s_credo_score"": 0.119,...",2025-07-16 12:45:12,2025-07-16,2025-07,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,3807091,00748d99-1df9-4a2a-b087-614aa9690c2f,60838070910018,beta_stack_model_sil,0.1146,"{""s_apps_score"": null, ""s_credo_score"": 0.1146...",2025-11-12 17:50:26,2025-11-12,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,ios
2,3613865,014f6d4b-45bc-43de-9549-44e4f3e5db25,60836138650017,beta_stack_model_sil,0.0636,"{""s_apps_score"": 0.3380516831572256, ""s_credo_...",2025-08-11 12:17:06,2025-08-11,2025-08,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
3,3080238,021f4021-0e2c-40fa-acab-abcae84ba09f,60830802380013,beta_stack_model_sil,0.1128,"{""s_apps_score"": 0.3686436943537751, ""s_credo_...",2025-09-08 11:47:39,2025-09-08,2025-09,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
4,3637258,025f73f7-f41c-4f91-acab-e81b9dd31f64,60836372580016,beta_stack_model_sil,0.1549,"{""s_apps_score"": 0.49779259935691195, ""s_credo...",2025-08-22 18:30:08,2025-08-22,2025-08,Prod,0,1,SIL Competitor,v1,Trench 2,Mall,android


In [17]:
df1 = dfd.copy()

### Train 

In [18]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
        deviceOs osType,
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd0 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (225063, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3206286,6b2d3c9c-efd4-4d62-a4d3-db28826ffd7a,60832062860017,beta_stack_model_sil,0.050984,"{""sb_demo_score"": 0.49638812817771, ""apps_scor...",2025-11-15 10:23:43,2025-01-21,2025-11,Dev_Test,0,1,SIL-Instore,v2,Trench 1,Appliance,android
1,2722179,00ada2bb-dff5-4e92-89fb-54726a784ac0,60827221790012,beta_stack_model_sil,0.096792,"{""sb_demo_score"": 0.48378518644819496, ""apps_s...",2024-08-04 18:50:42,2024-08-04,2024-08,Dev_Train,1,1,SIL ZERO,v2,Trench 1,Mobile,android
2,2794328,00dab900-0077-4d44-a4d2-9a268098124b,60827943280011,beta_stack_model_sil,0.134167,"{""sb_demo_score"": 0.31948796764442666, ""apps_s...",2024-08-26 14:37:34,2024-08-26,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
3,2807491,020ad01e-3335-4db2-a34b-691dbd9becac,60828074910013,beta_stack_model_sil,0.093304,"{""sb_demo_score"": 0.5900958497166336, ""apps_sc...",2024-08-30 14:26:41,2024-08-30,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
4,2726384,03a757e9-df4c-4940-abdc-b4ab95bfa43b,60827263840011,beta_stack_model_sil,0.058402,"{""sb_demo_score"": 0.549068221620335, ""apps_sco...",2024-08-06 11:09:07,2024-08-06,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android


In [19]:
df2 = dfd.copy()

### Concat 

In [20]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (265893, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3807091,00748d99-1df9-4a2a-b087-614aa9690c2f,60838070910018,beta_stack_model_sil,0.1146,"{""s_apps_score"": null, ""s_credo_score"": 0.1146...",2025-11-12 17:50:26,2025-11-12,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,ios
1,3788830,0cbaf1a7-c505-438f-b8d9-cd61ba62ca42,60837888300017,beta_stack_model_sil,0.1724,"{""s_apps_score"": 0.6614413141377518, ""s_credo_...",2025-11-03 13:33:19,2025-11-03,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
2,3743841,0e5f313e-f126-4ec7-9b3d-a120625cbd59,60837438410011,beta_stack_model_sil,0.1969,"{""s_apps_score"": 0.5735234884202352, ""s_credo_...",2025-10-14 14:32:11,2025-10-14,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
3,3765143,0f52d01e-2d0c-4f4b-96db-2f6331a883cc,60837651430019,beta_stack_model_sil,0.0900,"{""s_apps_score"": 0.3360637551152162, ""s_credo_...",2025-10-24 13:49:46,2025-10-24,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
4,3757095,18d90705-d7d7-460f-aba3-df3f56bb3970,60837570950011,beta_stack_model_sil,0.1003,"{""s_apps_score"": 0.4877888098391051, ""s_credo_...",2025-10-20 17:01:52,2025-10-20,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [21]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [22]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffpd10', 
    'FPD10',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
     ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

In [23]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (50222, 17)
The shape of the dimension table is:	 (1305, 12)


In [24]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ffd51a43-ce5b-43b5-9902-33a8e8f2ae77>

In [25]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b4b85d21-57ac-4cde-9373-fdc18dea961d>

## FPD30 

## Test 

In [26]:
sq = f"""with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
       deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()


The shape of the dataframe downloaded is:	 (86014, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3483974,0055929c-c763-46eb-8166-34d7e6ba3b70,60834839740014,beta_stack_model_sil,0.2100,"{""s_apps_score"": 0.562746465298505, ""s_credo_s...",2025-06-07 19:36:22,2025-06-07,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,3045243,00c388ac-c413-465a-9bfd-a566ebf0a7af,60830452430028,beta_stack_model_sil,0.1113,"{""s_apps_score"": 0.45345397973681784, ""s_credo...",2025-06-11 16:15:54,2025-06-11,2025-06,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,3606047,00faae6c-0df1-4a3c-9b78-a8ebd5d915d0,60836060470018,beta_stack_model_sil,0.0614,"{""s_apps_score"": 0.3358283343487627, ""s_credo_...",2025-08-07 19:13:02,2025-08-07,2025-08,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
3,3693376,0114a24c-3222-48d4-b302-819745d69d86,60836933760015,beta_stack_model_sil,0.0816,"{""s_apps_score"": 0.4050744995354494, ""s_credo_...",2025-09-19 19:18:10,2025-09-19,2025-09,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3573975,01db8c1c-3932-4fca-bda5-6a60e65b1582,60835739750016,beta_stack_model_sil,0.0908,"{""s_apps_score"": 0.5329688687163714, ""s_credo_...",2025-07-23 15:59:47,2025-07-23,2025-07,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android


In [27]:
df1 = dfd.copy()

### Train 

In [28]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
        deviceOs osType,
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (225063, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3206286,6b2d3c9c-efd4-4d62-a4d3-db28826ffd7a,60832062860017,beta_stack_model_sil,0.050984,"{""sb_demo_score"": 0.49638812817771, ""apps_scor...",2025-11-15 10:23:43,2025-01-21,2025-11,Dev_Test,0,1,SIL-Instore,v2,Trench 1,Appliance,android
1,2743666,00ecbe02-c535-426c-b52b-440500c61d6d,60827436660016,beta_stack_model_sil,0.063543,"{""sb_demo_score"": 0.3882950063889797, ""apps_sc...",2024-08-11 15:04:21,2024-08-11,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Mobile,android
2,2772964,030186c2-0e5a-458a-9473-253548fa577e,60827729640014,beta_stack_model_sil,0.145002,"{""sb_demo_score"": 0.6026284275164392, ""apps_sc...",2024-08-20 11:48:53,2024-08-20,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
3,2804788,0356efed-c550-413a-afff-f4bad81363e5,60828047880015,beta_stack_model_sil,0.076363,"{""sb_demo_score"": 0.7103103121756439, ""s_credo...",2024-08-29 16:11:03,2024-08-29,2024-08,Dev_Train,1,1,SIL-Instore,v2,Trench 1,Mobile,ios
4,2760912,03b01572-21cc-4815-89c0-11d6f2ec33bb,60827609120019,beta_stack_model_sil,0.146040,"{""sb_demo_score"": 0.6512224838900438, ""apps_sc...",2024-08-16 18:29:17,2024-08-16,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Mobile,android


In [29]:
df2 = dfd.copy()

### Concat 

In [30]:
df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (249015, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3731759,0393a476-985d-4c66-8e78-ac5b802ac11e,60837317590014,beta_stack_model_sil,0.0578,"{""s_apps_score"": 0.33478936841152895, ""s_credo...",2025-10-22 09:22:41,2025-10-22,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
1,3790997,1474568f-539d-4638-bf41-c5fb33374995,60837909970012,beta_stack_model_sil,0.0353,"{""s_apps_score"": 0.26965617799929115, ""s_credo...",2025-11-04 14:39:19,2025-11-04,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3791249,1d1abc68-6097-4fb1-98fe-4920055826ae,60837912490017,beta_stack_model_sil,0.1570,"{""s_apps_score"": 0.6238296226247549, ""s_credo_...",2025-11-04 16:54:03,2025-11-04,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3744333,1f223cb6-465a-46fa-b0f4-199735be0bbe,60837443330019,beta_stack_model_sil,0.2048,"{""s_apps_score"": 0.5403346169198664, ""s_credo_...",2025-10-14 17:02:34,2025-10-14,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
4,3760555,220ad68b-4900-444a-9595-2bac0e441a09,60837605550015,beta_stack_model_sil,0.0676,"{""s_apps_score"": 0.33464726590508703, ""s_credo...",2025-10-22 13:39:45,2025-10-22,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android


In [31]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [32]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffpd30', 
    'FPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType',   
    account_id_column='digitalLoanAccountId'
)

In [33]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (48654, 17)
The shape of the dimension table is:	 (1305, 12)


In [34]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=665d8dbb-3229-4287-8714-701c963bb906>

In [35]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=041065c9-9b78-4159-a289-b9cfd7ca48d4>

## FSPD30 

## Test 

In [36]:
sq = f"""with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
        deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fspd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()


The shape of the dataframe downloaded is:	 (67023, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3501806,007b9e45-0cae-41ae-b204-587619fedd35,60835018060017,beta_stack_model_sil,0.0632,"{""s_apps_score"": 0.4992307173514058, ""s_credo_...",2025-06-16 18:32:28,2025-06-16,2025-06,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
1,3467060,01f71dab-b8fc-4f18-9a28-c67e09e25011,60834670600017,beta_stack_model_sil,0.0613,"{""s_apps_score"": 0.4154993836551738, ""s_credo_...",2025-05-30 10:53:50,2025-05-30,2025-05,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
2,3645879,029f1ca7-30d5-4e9c-a226-6a03dd661ec5,60836458790016,beta_stack_model_sil,0.0962,"{""s_apps_score"": 0.40064402172245583, ""s_credo...",2025-08-27 10:40:50,2025-08-27,2025-08,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3028706,0391a06c-11cd-42dc-a935-443e3b8dc892,60830287060021,beta_stack_model_sil,0.1428,"{""s_apps_score"": 0.5014336333802343, ""s_credo_...",2025-09-25 15:33:10,2025-09-25,2025-09,Prod,0,1,SIL Competitor,v1,Trench 3,Appliance,android
4,3766003,0f68ab78-9868-4fe8-a403-57af35728c07,60837660030012,beta_stack_model_sil,0.1144,"{""s_apps_score"": 0.600114969880684, ""s_credo_s...",2025-10-24 18:48:14,2025-10-24,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android


In [37]:
df1 = dfd.copy()

### Train 

In [38]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
      deviceOs osType,
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  deffspd30,
  flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and flg_mature_fspd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (225063, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3206286,6b2d3c9c-efd4-4d62-a4d3-db28826ffd7a,60832062860017,beta_stack_model_sil,0.050984,"{""sb_demo_score"": 0.49638812817771, ""apps_scor...",2025-11-15 10:23:43,2025-01-21,2025-11,Dev_Test,0,1,SIL-Instore,v2,Trench 1,Appliance,android
1,2757130,0182f8b3-39d2-4e96-a04a-a401e834c594,60827571300011,beta_stack_model_sil,0.219407,"{""sb_demo_score"": 0.5217332318652251, ""apps_sc...",2024-08-15 17:19:17,2024-08-15,2024-08,Dev_Train,1,1,SIL-Instore,v2,Trench 1,Appliance,android
2,2725023,0332e54e-2242-4a4b-a4f7-c5a351ab9dd1,60827250230015,beta_stack_model_sil,0.404174,"{""sb_demo_score"": 0.5348218016961699, ""apps_sc...",2024-08-05 18:17:02,2024-08-05,2024-08,Dev_Train,1,1,SIL-Instore,v2,Trench 1,Appliance,android
3,2770184,03d8a8f8-5a92-44fa-a982-600eba2cfdfe,60827701840016,beta_stack_model_sil,0.068525,"{""sb_demo_score"": 0.7317250173740618, ""apps_sc...",2024-08-19 14:13:41,2024-08-19,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
4,2762265,0471662f-969c-4f35-84e5-e4f06b6d0008,60827622650019,beta_stack_model_sil,0.055683,"{""sb_demo_score"": 0.27425514900289455, ""apps_s...",2024-08-17 10:25:03,2024-08-17,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android


In [39]:
df2 = dfd.copy()

### Concat 

In [40]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (230024, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3766003,0f68ab78-9868-4fe8-a403-57af35728c07,60837660030012,beta_stack_model_sil,0.1144,"{""s_apps_score"": 0.600114969880684, ""s_credo_s...",2025-10-24 18:48:14,2025-10-24,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
1,3763526,18b896ec-52bf-4b99-81f4-65dcb6a6de76,60837635260012,beta_stack_model_sil,0.1836,"{""s_apps_score"": 0.6331156753829933, ""s_credo_...",2025-10-23 17:29:31,2025-10-23,2025-10,Prod,1,1,SIL Competitor,v1,Trench 2,Appliance,android
2,3761516,345a0116-dec1-443c-9ed7-bc9d89b37a2b,60837615160012,beta_stack_model_sil,0.0784,"{""s_apps_score"": 0.6152961365294347, ""s_credo_...",2025-10-22 19:13:39,2025-10-22,2025-10,Prod,1,1,SIL Competitor,v1,Trench 2,Mall,android
3,3763506,3fc1d34f-093a-4235-a12f-6482deed8ae0,60837635060018,beta_stack_model_sil,0.1762,"{""s_apps_score"": 0.5348163615761626, ""s_credo_...",2025-10-23 17:05:12,2025-10-23,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3760883,43ccea66-b551-4402-93c7-7b7a4f513e6f,60837608830018,beta_stack_model_sil,0.1723,"{""s_apps_score"": 0.5653247650218304, ""s_credo_...",2025-10-22 15:22:03,2025-10-22,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [41]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [42]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffspd30', 
    'FSPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
     ostype_column='osType',  
    account_id_column='digitalLoanAccountId'
)

In [43]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (46310, 17)
The shape of the dimension table is:	 (1305, 12)


In [44]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0718f43c-21cc-44f5-9c61-fe5b10787787>

In [45]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0e2b7eef-f803-4b06-893c-47a02135f801>

## FSTPD30 

## Test 

In [46]:
sq = f"""with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
        deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()


The shape of the dataframe downloaded is:	 (52793, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3704120,06d88b7a-a18d-4e05-922b-9d5e9bb386ea,60837041200018,beta_stack_model_sil,0.0622,"{""s_apps_score"": 0.4868402678514773, ""s_credo_...",2025-09-25 12:10:31,2025-09-25,2025-09,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
1,3406883,0b106f5a-bb1d-46f5-9fa3-f6aa3e187342,60834068830014,beta_stack_model_sil,0.0616,"{""s_apps_score"": 0.4382645688665238, ""s_credo_...",2025-04-28 12:44:31,2025-04-28,2025-04,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3471566,0f257361-93fc-4ecf-b27f-06ec57c5d747,60834715660019,beta_stack_model_sil,0.0615,"{""s_apps_score"": 0.36803753805632106, ""s_credo...",2025-06-01 13:41:50,2025-06-01,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
3,3446176,293dc668-f238-4883-b4cc-3aefc09f29e8,60834461760014,beta_stack_model_sil,0.1545,"{""s_apps_score"": null, ""s_credo_score"": 0.1545...",2025-05-19 16:05:43,2025-05-19,2025-05,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,ios
4,3473302,2bd52b35-befd-415f-8b1e-a9e022849be5,60834733020015,beta_stack_model_sil,0.1538,"{""s_apps_score"": 0.5389457256143597, ""s_credo_...",2025-06-02 10:03:28,2025-06-02,2025-06,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android


In [47]:
df1 = dfd.copy()

### Train 

In [48]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection,
        deviceOs osType,
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), 
  cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.s_credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  deffstpd30,
  flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and flg_mature_fstpd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (215593, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3206286,6b2d3c9c-efd4-4d62-a4d3-db28826ffd7a,60832062860017,beta_stack_model_sil,0.050984,"{""sb_demo_score"": 0.49638812817771, ""apps_scor...",2025-11-15 10:23:43,2025-01-21,2025-11,Dev_Test,0,1,SIL-Instore,v2,Trench 1,Appliance,android
1,2808536,0118cc26-a9ae-4d13-9ada-33e866f261f6,60828085360013,beta_stack_model_sil,0.173331,"{""sb_demo_score"": 0.4213222811289653, ""apps_sc...",2024-08-30 18:42:24,2024-08-30,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
2,2720833,01e2edb8-8886-40fe-aaf0-fa48ba916406,60827208330011,beta_stack_model_sil,0.099382,"{""sb_demo_score"": 0.3446027674260736, ""apps_sc...",2024-08-04 14:12:37,2024-08-04,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
3,2727354,02653616-26ea-47a8-9fe1-c1f978be799d,60827273540019,beta_stack_model_sil,0.186241,"{""sb_demo_score"": 0.658903148454277, ""apps_sco...",2024-08-06 15:19:27,2024-08-06,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Appliance,android
4,2798814,05419431-9acc-499d-b36c-962e5b548c58,60827988140015,beta_stack_model_sil,0.087437,"{""sb_demo_score"": 0.29639916456811133, ""apps_s...",2024-08-27 18:54:28,2024-08-27,2024-08,Dev_Train,0,1,SIL-Instore,v2,Trench 1,Mobile,android


In [49]:
df2 = dfd.copy()

### Concat 

In [50]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (215774, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3644353,dd8b0a2f-d956-43f9-abaa-8127f9382ed3,60836443530017,beta_stack_model_sil,0.1371,"{""s_apps_score"": 0.6528019583726831, ""s_credo_...",2025-08-26 11:52:39,2025-08-26,2025-08,Prod,0,1,SIL Competitor,v1,Trench 2,Mall,android
1,3339905,ca1cdce8-a27f-497a-8032-726079179be6,60833399050014,beta_stack_model_sil,0.1348,"{""s_apps_score"": 0.5180843963444356, ""s_credo_...",2025-03-25 14:59:59,2025-03-28,2025-03,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3634235,6d640f14-ffd2-43a3-b1e4-62ec3e82b5d9,60836342350017,beta_stack_model_sil,0.0889,"{""s_apps_score"": 0.37506613452655846, ""s_credo...",2025-08-26 16:43:52,2025-08-26,2025-08,Prod,1,1,SIL Competitor,v1,Trench 2,Mall,android
3,3435452,c9080f44-1a0b-4e44-aeb4-0853679519fc,60834354520016,beta_stack_model_sil,0.1450,"{""s_apps_score"": 0.6072078321205403, ""s_credo_...",2025-05-13 17:54:17,2025-05-13,2025-05,Prod,1,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3343551,fd3767af-fd32-42d6-b702-0253dd90e03d,60833435510019,beta_stack_model_sil,0.1834,"{""s_apps_score"": 0.6072078321205403, ""s_credo_...",2025-03-27 16:08:12,2025-03-27,2025-03,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android


In [51]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [52]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffstpd30', 
    'FSTPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
            ostype_column='osType',   
    account_id_column='digitalLoanAccountId'
)

In [53]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (42888, 17)
The shape of the dimension table is:	 (1236, 12)


In [54]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9853a0da-717e-4885-a9e4-0ce84c8d845a>

In [55]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=15482502-c589-4a9c-b3a1-4b6f55d27154>